In [15]:
# with open('wiki.train.raw', 'r', encoding='utf-8') as file:
#     data = file.read()
#     print(data)
#     print(type(data))


In [20]:
import nltk
import wikidata

In [2]:
from transformers import RobertaTokenizer

# Load the pre-trained 'roberta-base' tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Now you can use the tokenizer
text = "Hello, world! i am here to test the functions of this tokenizer such that it helps me."
encoded_input = tokenizer(text, return_tensors='pt')

print(encoded_input)


{'input_ids': tensor([[    0, 31414,     6,   232,   328,   939,   524,   259,     7,  1296,
             5,  8047,     9,    42, 19233,  6315,   215,    14,    24,  2607,
           162,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [3]:
tokens= tokenizer.tokenize(text)
print(tokens)

['Hello', ',', 'Ġworld', '!', 'Ġi', 'Ġam', 'Ġhere', 'Ġto', 'Ġtest', 'Ġthe', 'Ġfunctions', 'Ġof', 'Ġthis', 'Ġtoken', 'izer', 'Ġsuch', 'Ġthat', 'Ġit', 'Ġhelps', 'Ġme', '.']


In [4]:
def build_vocab(tokens, add_unknown=True):
    vocab = set(tokens)
    if add_unknown:
        vocab.add('<UNK>')
    token_to_index = {token: idx for idx, token in enumerate(vocab)}
    index_to_token = {idx: token for token, idx in token_to_index.items()}
    return token_to_index, index_to_token

token_to_index, index_to_token= build_vocab(tokens, add_unknown=True)
print(token_to_index)
print(".................................")
print(index_to_token)

{'!': 0, 'Ġthis': 1, 'Ġi': 2, 'Ġthe': 3, 'Ġworld': 4, 'Ġhelps': 5, 'Ġthat': 6, 'Ġtoken': 7, 'Ġam': 8, 'izer': 9, 'Ġto': 10, 'Ġit': 11, 'Hello': 12, 'Ġof': 13, ',': 14, 'Ġfunctions': 15, 'Ġhere': 16, 'Ġsuch': 17, 'Ġtest': 18, '.': 19, 'Ġme': 20, '<UNK>': 21}
.................................
{0: '!', 1: 'Ġthis', 2: 'Ġi', 3: 'Ġthe', 4: 'Ġworld', 5: 'Ġhelps', 6: 'Ġthat', 7: 'Ġtoken', 8: 'Ġam', 9: 'izer', 10: 'Ġto', 11: 'Ġit', 12: 'Hello', 13: 'Ġof', 14: ',', 15: 'Ġfunctions', 16: 'Ġhere', 17: 'Ġsuch', 18: 'Ġtest', 19: '.', 20: 'Ġme', 21: '<UNK>'}


In [5]:
# Function to build a vocabulary from tokens
def build_vocab(tokens):
    vocab = set(tokens)
    vocab.add('<UNK>')  # Add a special token for unknown words
    token_to_index = {token: idx for idx, token in enumerate(vocab)}
    index_to_token = {idx: token for token, idx in token_to_index.items()}
    return token_to_index, index_to_token


In [11]:
# Build the vocabulary
token_to_index, index_to_token = build_vocab(tokens)

In [6]:
vocab_size = len(token_to_index)

In [7]:
print(vocab_size)

22


In [8]:
# Function to convert tokens to indices
def tokens_to_indices(tokens, token_to_index):
    return [token_to_index[token] for token in tokens]

In [9]:
# Function to create sequences for training
def create_sequences(token_indices, seq_length):
    sequences = [token_indices[i:i + seq_length] for i in range(len(token_indices) - seq_length)]
    return sequences

token_indices= tokens_to_indices(tokens, token_to_index)
# sequencess= create_sequences(token_indices, seq_length=5)
print(token_indices)

[12, 14, 4, 0, 2, 8, 16, 10, 18, 3, 15, 13, 1, 7, 9, 17, 6, 11, 5, 20, 19]


In [12]:
def create_sequences(token_indices, seq_length):
    sequences = [token_indices[i:i + seq_length] for i in range(len(token_indices) - seq_length)]
    return sequences

sequences= create_sequences(token_indices, seq_length=5)
print(sequences)

[[12, 14, 4, 0, 2], [14, 4, 0, 2, 8], [4, 0, 2, 8, 16], [0, 2, 8, 16, 10], [2, 8, 16, 10, 18], [8, 16, 10, 18, 3], [16, 10, 18, 3, 15], [10, 18, 3, 15, 13], [18, 3, 15, 13, 1], [3, 15, 13, 1, 7], [15, 13, 1, 7, 9], [13, 1, 7, 9, 17], [1, 7, 9, 17, 6], [7, 9, 17, 6, 11], [9, 17, 6, 11, 5], [17, 6, 11, 5, 20]]


In [14]:
# Function to prepare training data
def prepare_training_data(sequences, seq_length):
    # Ensure that inputs and targets are of the same length
    inputs = [sequence[:-1] for sequence in sequences]  # Exclude the last token for input
    targets = [sequence[1:] for sequence in sequences]  # Exclude the first token for target
    return inputs, targets


inputs, targets= prepare_training_data(sequences, seq_length=5)
print(inputs)
print("......................")
print(targets)

[[12, 14, 4, 0], [14, 4, 0, 2], [4, 0, 2, 8], [0, 2, 8, 16], [2, 8, 16, 10], [8, 16, 10, 18], [16, 10, 18, 3], [10, 18, 3, 15], [18, 3, 15, 13], [3, 15, 13, 1], [15, 13, 1, 7], [13, 1, 7, 9], [1, 7, 9, 17], [7, 9, 17, 6], [9, 17, 6, 11], [17, 6, 11, 5]]
......................
[[14, 4, 0, 2], [4, 0, 2, 8], [0, 2, 8, 16], [2, 8, 16, 10], [8, 16, 10, 18], [16, 10, 18, 3], [10, 18, 3, 15], [18, 3, 15, 13], [3, 15, 13, 1], [15, 13, 1, 7], [13, 1, 7, 9], [1, 7, 9, 17], [7, 9, 17, 6], [9, 17, 6, 11], [17, 6, 11, 5], [6, 11, 5, 20]]


In [14]:
import pickle

In [15]:
# Save token_to_index and index_to_token
with open('bpe_token_to_index.pkl', 'wb') as f:
    pickle.dump(token_to_index, f)

with open('bpe_index_to_token.pkl', 'wb') as f:
    pickle.dump(index_to_token, f)